In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
movies_companies_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("companyId", IntegerType(), True),
])

In [0]:
movies_companies_df = spark.read \
    .schema(movies_companies_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/movie_company")

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
display(movies_companies_df)

In [0]:
movies_companies_renamed_df = movies_companies_df \
    .withColumnRenamed("movieId", "movie_id") \
    .withColumnRenamed("companyId", "company_id") \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("environment", lit(v_environment))\
    .withColumn("file_date", lit(v_file_date))

In [0]:
display(movies_companies_renamed_df)

In [0]:
#overwrite_partition(movies_companies_renamed_df, "movie_silver", "movie_company")

In [0]:
merge_condicion = "tgt.company_id = src.company_id AND tgt.movie_id = src.movie_id AND tgt.file_date = src.file_date"
merge_delta_lake(movies_companies_renamed_df, "movie_silver", "movie_company", silver_folder_path, merge_condicion, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.movie_company
GROUP BY file_date;

In [0]:
%sql
SELECT * FROM movie_silver.movie_company